In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, max, min, mean

In [3]:
spark = SparkSession.builder.appName("Exercises Modeles AutoHeberge").getOrCreate()

24/12/18 12:47:23 WARN Utils: Your hostname, MacBook-Air-de-Aya.local resolves to a loopback address: 127.0.0.1; using 192.168.1.63 instead (on interface en0)
24/12/18 12:47:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/18 12:47:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# --- Exercice 1: Analyse des ventes ---
pathCSVVentes = "/Users/ayaelyaagoubi/Downloads/ventes.csv"
ventes = spark.read.csv(pathCSVVentes, header=True, inferSchema=True)
ventes = ventes.withColumn("Chiffre_d_affaires", col("Quantité") * col("Prix_unitaire"))
chiffre_affaires_total = ventes.agg(sum("Chiffre_d_affaires").alias("Chiffre_affaires_total")).collect()[0][0]

produit_plus_vendu = ventes.groupBy("Produit").sum("Quantité").withColumnRenamed("sum(Quantité)", "Total_Quantité") \
    .orderBy(col("Total_Quantité").desc()).first()

print(f"Chiffre d'affaires total : {chiffre_affaires_total} ")
print(f"Produit le plus vendu : {produit_plus_vendu['Produit']} ({produit_plus_vendu['Total_Quantité']} unités)")


Chiffre d'affaires total : 2710 
Produit le plus vendu : Souris (10 unités)


In [6]:
# --- Exercice 2: Analyse des utilisateurs ---
pathJSONUtilisateurs =  "/Users/ayaelyaagoubi/Downloads/utilisateurs.json"
utilisateurs = spark.read.json(pathJSONUtilisateurs )

age_moyen = utilisateurs.select(avg("âge").alias("Âge_moyen")).collect()[0][0]

utilisateurs_par_ville = utilisateurs.groupBy("ville").count()

plus_jeune_utilisateur = utilisateurs.orderBy(col("âge").asc()).first()

print(f"Âge moyen : {age_moyen} ans")
utilisateurs_par_ville.show()
print(f"Plus jeune utilisateur : {plus_jeune_utilisateur['nom']} ({plus_jeune_utilisateur['âge']} ans)")


Âge moyen : 30.4 ans
+---------+-----+
|    ville|count|
+---------+-----+
|Marseille|    1|
|    Paris|    2|
|     Lyon|    2|
+---------+-----+

Plus jeune utilisateur : Emma (22 ans)


In [9]:
# --- Exercice 3: Nettoyage de données ---
pathCSVClients = "/Users/ayaelyaagoubi/Downloads/clients.csv"
clients = spark.read.csv(pathCSVClients, header=True, inferSchema=True)

age_moyen = clients.select(mean("Âge").alias("moyenne_age")).collect()[0][0]

clients = clients.fillna({"Âge": age_moyen})

clients = clients.fillna({"Ville": "Inconnue"})

clients = clients.filter(col("Revenu").isNotNull())

clients.show()

+------+----+---------+-------+
|   Nom| Âge|    Ville| Revenu|
+------+----+---------+-------+
| Alice|25.0|    Paris|50000.0|
|   Bob|30.5|     Lyon|40000.0|
|Claire|35.0|Marseille|35000.0|
| David|40.0| Inconnue|45000.0|
+------+----+---------+-------+



In [10]:
# --- Exercice 4: Produits les plus chers ---
pathCSVProduits = "/Users/ayaelyaagoubi/Downloads/produits.csv"
produits = spark.read.csv(pathCSVProduits, header=True, inferSchema=True)

produits_plus_chers = produits.orderBy(col("Prix").desc()).limit(3)

produits_plus_chers.show()


+----------+------------+----+
|   Produit|   Catégorie|Prix|
+----------+------------+----+
|Ordinateur|Électronique| 800|
|     Table|      Bureau| 150|
|Imprimante|Électronique| 120|
+----------+------------+----+



In [11]:
# --- Exercice 5: Analyse des transactions ---
pathCSVTransaction= "/Users/ayaelyaagoubi/Downloads/transactions.csv"

transactions = spark.read.csv(pathCSVTransaction, header=True, inferSchema=True)

depenses_totales = transactions.groupBy("Client").sum("Montant").withColumnRenamed("sum(Montant)", "Dépenses_totales")

client_plus_depense = depenses_totales.orderBy(col("Dépenses_totales").desc()).first()

depenses_totales.show()
print(f"Client ayant dépensé le plus : {client_plus_depense['Client']} ({client_plus_depense['Dépenses_totales']} €)")

+------+----------------+
|Client|Dépenses_totales|
+------+----------------+
|   Bob|             200|
| Alice|             250|
|  Emma|             350|
| David|             250|
+------+----------------+

Client ayant dépensé le plus : Emma (350 €)


In [12]:
# --- Exercice 6: Agrégation de données par catégorie ---
pathCSVProduits = "/Users/ayaelyaagoubi/Downloads/produits.csv"

produits = spark.read.csv(pathCSVProduits, header=True, inferSchema=True)

statistiques_categorie = produits.groupBy("Catégorie").agg(
    avg("Prix").alias("Prix_moyen"),
    sum("Prix").alias("Prix_total")
)

statistiques_categorie.show()

+------------+----------+----------+
|   Catégorie|Prix_moyen|Prix_total|
+------------+----------+----------+
|Électronique|    238.75|       955|
|      Bureau|     115.0|       230|
+------------+----------+----------+

